In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np

In [0]:
df_quora = pd.read_csv('/content/drive/My Drive/Text Project/Quora.csv')
df_quora.fillna('.',inplace = True)
df_quora['Comments'] = df_quora['Field1'] + df_quora['Field2'] + df_quora['Field2'] + df_quora['Field4'] + df_quora['Field5'] + df_quora['Field6']

In [0]:
df_quora = df_quora.drop(['Field1'],axis=1)
df_quora = df_quora.drop(['Field2'],axis=1)
df_quora = df_quora.drop(['Field3'],axis=1)
df_quora = df_quora.drop(['Field4'],axis=1)
df_quora = df_quora.drop(['Field5'],axis=1)
df_quora = df_quora.drop(['Field6'],axis=1)

In [5]:
df_quora.head()

,Comments
0,People who have experienced both US healthcare...
1,"As a European, would you prefer the US healthc..."
2,If you've lived under both the US healthcare s...
3,If universal healthcare systems of Europe are ...
4,How much does healthcare cost the individual i...


In [6]:
# Import the Data Frame
import nltk
nltk.download('stopwords')
nltk.download('punkt')
# nltk.download('all')
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set( stopwords.words('english'))
from spacy.lang.en import English
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import spacy 
import numpy as np
import pandas as pd

# Function to remove stopwords and replacing model names with brand names
# This function will be applied to all comments in our data frame

def filterReview(review):
    nlp = English()
    tokens = word_tokenize(review)
    word_str = ""

    # Removing numbers
    for w in tokens:
        try:
            float(w)
        
        # If not a number, we will consider the words
        except Exception as e:
            if w.lower() not in stop_words: 
                word_str = word_str + " " + w.lower()
        continue
    
    # Lemmatization
    lemmatized = []
    doc = nlp(' '.join(word_str.split()))
    for word in doc:
        lemmatized.append(word.lemma_)
    
    return lemmatized

# Backup fundtion to clean the data 
def removePunctuations(review):
    nlp = English()
    tokens = word_tokenize(review)
    stop_words_punct = set(stopwords.words('english') + list(punctuation))
    no_stopwords = [w.lower() for w in tokens if w.lower() not in stop_words_punct]
    word_str = ""
    for w in no_stopwords:
        # Removing numbers
        try:
            float(w)
        except Exception as e:
            word_str = word_str + " " + w
        continue
    
    # Lemmatization
    lemmatized = []
    doc = nlp(' '.join(word_str.split()))
    for word in doc:
        lemmatized.append(word.lemma_)
    
    return lemmatized

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
df_quora['Filter Comments NO punct'] = df_quora['Comments'].map(removePunctuations)
# df_redUS['Filtered Comments'] = df_redUS['Comments'].map(filterReview)

In [8]:
df_quora.head()

,Comments,Filter Comments NO punct
0,People who have experienced both US healthcare...,"[people, experience, us, healthcare, non, -, u..."
1,"As a European, would you prefer the US healthc...","[european, would, prefer, us, healthcare, syst..."
2,If you've lived under both the US healthcare s...,"[', have, live, us, healthcare, system, health..."
3,If universal healthcare systems of Europe are ...,"[universal, healthcare, system, europe, great,..."
4,How much does healthcare cost the individual i...,"[much, healthcare, cost, individual, country, ..."


## Topic Modelling

In [0]:
import gensim
from gensim.utils import simple_preprocess
from gensim.test.utils import common_corpus, common_dictionary

In [0]:
dictionary_quora = gensim.corpora.Dictionary(df_quora['Filter Comments NO punct'])
dictionary_quora.filter_extremes(no_below=1, no_above=0.7) #, keep_n=100000)
bow_corpus_before = [dictionary_quora.doc2bow(words) for words in df_quora['Filter Comments NO punct']]
lda_model_quora = gensim.models.LdaMulticore(bow_corpus_before, num_topics=6, id2word=dictionary_quora, passes=2, workers=2, chunksize=100, random_state=1000)

In [11]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(lda_model_quora.print_topics())

[   (   0,
        '0.017*"\'" + 0.012*"not" + 0.012*"company" + 0.011*"bill" + '
        '0.010*"lab" + 0.010*"month" + 0.009*"call" + 0.009*"various" + '
        '0.008*"plan" + 0.008*"tell"'),
    (   1,
        '0.016*"care" + 0.011*"cost" + 0.011*"european" + 0.010*"government" + '
        '0.008*"people" + 0.008*"”" + 0.008*"“" + 0.007*"patient" + '
        '0.006*"americans" + 0.006*"service"'),
    (   2,
        '0.017*"care" + 0.010*"name" + 0.010*"focus" + 0.008*"country" + '
        '0.008*"diagnosis" + 0.008*"pocket" + 0.007*"receive" + '
        '0.007*"payment" + 0.007*"bill" + 0.006*"canada"'),
    (   3,
        '0.008*"surgery" + 0.008*"work" + 0.007*"specialist" + 0.006*"visit" + '
        '0.005*"two" + 0.005*"day" + 0.005*"nhs" + 0.005*"multiple" + '
        '0.005*"fantastic" + 0.005*"accident"'),
    (   4,
        '0.016*"care" + 0.013*"people" + 0.013*"canada" + 0.013*"cost" + '
        '0.008*"year" + 0.007*"treatment" + 0.007*"never" + 0.007*"work" + '
      

In [12]:
lda_model_quora = gensim.models.LdaMulticore(bow_corpus_before, num_topics=5, id2word=dictionary_quora, passes=10, workers=2, chunksize=100, random_state=1000)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(lda_model_quora.print_topics())

[   (   0,
        '0.013*"\'" + 0.011*"company" + 0.011*"bill" + 0.009*"month" + '
        '0.009*"year" + 0.009*"call" + 0.009*"late" + 0.008*"not" + '
        '0.007*"back" + 0.007*"experience"'),
    (   1,
        '0.009*"surgery" + 0.009*"work" + 0.009*"specialist" + 0.006*"nhs" + '
        '0.006*"two" + 0.006*"visit" + 0.006*"multiple" + 0.006*"employer" + '
        '0.006*"fantastic" + 0.006*"accident"'),
    (   2,
        '0.019*"care" + 0.013*"name" + 0.013*"focus" + 0.011*"diagnosis" + '
        '0.010*"country" + 0.010*"pocket" + 0.009*"receive" + 0.009*"payment" '
        '+ 0.008*"bill" + 0.008*"uk"'),
    (   3,
        '0.010*"nurse" + 0.009*"year" + 0.009*"canada" + 0.008*"“" + 0.008*"”" '
        '+ 0.007*"work" + 0.006*"`" + 0.006*"people" + 0.006*"month" + '
        '0.006*"cost"'),
    (   4,
        '0.020*"care" + 0.013*"cost" + 0.012*"people" + 0.009*"european" + '
        '0.008*"government" + 0.007*"patient" + 0.007*"”" + 0.007*"“" + '
        '0.007*"work" 

## Topic Modelling after removing punctuations

In [0]:
dictionary_quora = gensim.corpora.Dictionary(df_quora['Filter Comments NO punct'])
dictionary_quora.filter_extremes(no_below=1, no_above=0.7) #, keep_n=100000)
bow_corpus_before = [dictionary_quora.doc2bow(words) for words in df_quora['Filter Comments NO punct']]
lda_model_quora = gensim.models.LdaMulticore(bow_corpus_before, num_topics=5, id2word=dictionary_quora, passes=10, workers=2, chunksize=100, random_state=1000)

In [14]:
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(lda_model_quora.print_topics())

[   (   0,
        '0.013*"\'" + 0.011*"company" + 0.011*"bill" + 0.009*"month" + '
        '0.009*"year" + 0.009*"call" + 0.009*"late" + 0.008*"not" + '
        '0.007*"back" + 0.007*"experience"'),
    (   1,
        '0.009*"surgery" + 0.009*"work" + 0.009*"specialist" + 0.006*"nhs" + '
        '0.006*"two" + 0.006*"visit" + 0.006*"multiple" + 0.006*"employer" + '
        '0.006*"fantastic" + 0.006*"accident"'),
    (   2,
        '0.019*"care" + 0.013*"name" + 0.013*"focus" + 0.011*"diagnosis" + '
        '0.010*"country" + 0.010*"pocket" + 0.009*"receive" + 0.009*"payment" '
        '+ 0.008*"bill" + 0.008*"uk"'),
    (   3,
        '0.010*"nurse" + 0.009*"year" + 0.009*"canada" + 0.008*"“" + 0.008*"”" '
        '+ 0.007*"work" + 0.006*"`" + 0.006*"people" + 0.006*"month" + '
        '0.006*"cost"'),
    (   4,
        '0.020*"care" + 0.013*"cost" + 0.012*"people" + 0.009*"european" + '
        '0.008*"government" + 0.007*"patient" + 0.007*"”" + 0.007*"“" + '
        '0.007*"work" 

In [15]:
lda_model_quora = gensim.models.LdaMulticore(bow_corpus_before, num_topics=5, id2word=dictionary_quora, passes=30, workers=2, chunksize=100, random_state=1000)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(lda_model_quora.print_topics())

[   (   0,
        '0.013*"\'" + 0.011*"company" + 0.011*"bill" + 0.009*"month" + '
        '0.009*"call" + 0.009*"year" + 0.009*"late" + 0.008*"not" + '
        '0.007*"back" + 0.007*"experience"'),
    (   1,
        '0.010*"surgery" + 0.009*"work" + 0.009*"specialist" + 0.006*"two" + '
        '0.006*"visit" + 0.006*"nhs" + 0.006*"multiple" + 0.006*"employer" + '
        '0.006*"accident" + 0.006*"fantastic"'),
    (   2,
        '0.019*"care" + 0.013*"focus" + 0.013*"name" + 0.011*"diagnosis" + '
        '0.010*"country" + 0.010*"pocket" + 0.010*"receive" + 0.010*"payment" '
        '+ 0.008*"uk" + 0.008*"bill"'),
    (   3,
        '0.010*"nurse" + 0.009*"canada" + 0.009*"year" + 0.008*"“" + 0.008*"”" '
        '+ 0.007*"cost" + 0.007*"people" + 0.007*"work" + 0.006*"`" + '
        '0.006*"month"'),
    (   4,
        '0.020*"care" + 0.013*"cost" + 0.012*"people" + 0.009*"european" + '
        '0.008*"government" + 0.007*"patient" + 0.007*"”" + 0.007*"“" + '
        '0.007*"work" 

## Additional Stop Words removal

In [0]:
new_stopword_list = ['gt', '`', '-', '“', '-', '/', 'amp', '•', 'subreddit', '’', '\'s',"\'\'", "\'", '”', 'argument', 'amitheasshole', 'https', 'http']
def additionalStop(comment):
    return [w for w in comment if len(w)>3 and w not in new_stopword_list]

In [0]:
df_quora['filter V1'] = df_quora['Filter Comments NO punct'].map(additionalStop)

In [0]:
dictionary_quora = gensim.corpora.Dictionary(df_quora['filter V1'])
dictionary_quora.filter_extremes(no_below=1, no_above=0.7) #, keep_n=100000)
bow_corpus_before = [dictionary_quora.doc2bow(words) for words in df_quora['filter V1']]

In [0]:
lda_model_quora = gensim.models.LdaMulticore(bow_corpus_before, num_topics=5, id2word=dictionary_quora, passes=10, workers=2, chunksize=100, random_state=1000)

In [20]:
lda_model_quora = gensim.models.LdaMulticore(bow_corpus_before, num_topics=3, id2word=dictionary_quora, passes=10, workers=2, chunksize=100, random_state=1000)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(lda_model_quora.print_topics())

[   (   0,
        '0.014*"care" + 0.012*"company" + 0.012*"people" + 0.011*"cost" + '
        '0.010*"bill" + 0.010*"plan" + 0.007*"month" + 0.007*"never" + '
        '0.006*"know" + 0.006*"call"'),
    (   1,
        '0.012*"nurse" + 0.010*"care" + 0.006*"work" + 0.006*"grant" + '
        '0.006*"pain" + 0.006*"name" + 0.006*"focus" + 0.005*"take" + '
        '0.005*"university" + 0.005*"diagnosis"'),
    (   2,
        '0.011*"care" + 0.011*"cost" + 0.009*"year" + 0.008*"canada" + '
        '0.008*"people" + 0.007*"european" + 0.006*"treatment" + '
        '0.006*"surgery" + 0.006*"work" + 0.006*"month"')]


In [21]:
lda_model_quora = gensim.models.LdaMulticore(bow_corpus_before, num_topics=5, id2word=dictionary_quora, passes=10, workers=2, chunksize=100, random_state=1000)
lda_model_quora.print_topics()

[(0,
  '0.016*"care" + 0.014*"company" + 0.013*"people" + 0.013*"cost" + 0.011*"bill" + 0.011*"plan" + 0.008*"month" + 0.008*"never" + 0.007*"know" + 0.007*"call"'),
 (1,
  '0.021*"care" + 0.015*"name" + 0.015*"focus" + 0.013*"diagnosis" + 0.012*"country" + 0.012*"pocket" + 0.011*"receive" + 0.011*"payment" + 0.008*"bill" + 0.008*"triagedoctor"'),
 (2,
  '0.014*"care" + 0.014*"cost" + 0.011*"year" + 0.010*"canada" + 0.009*"people" + 0.009*"european" + 0.007*"government" + 0.007*"treatment" + 0.006*"month" + 0.006*"country"'),
 (3,
  '0.013*"nurse" + 0.011*"work" + 0.006*"grant" + 0.005*"year" + 0.005*"take" + 0.005*"university" + 0.005*"surgery" + 0.005*"pain" + 0.005*"like" + 0.005*"specialist"'),
 (4,
  '0.001*"care" + 0.001*"year" + 0.001*"canada" + 0.001*"people" + 0.001*"nurse" + 0.001*"bill" + 0.001*"month" + 0.001*"cost" + 0.001*"country" + 0.001*"take"')]

In [22]:
lda_model_quora = gensim.models.LdaMulticore(bow_corpus_before, num_topics=5, id2word=dictionary_quora, passes=50, workers=2, chunksize=100, random_state=1000)
lda_model_quora.print_topics()

[(0,
  '0.016*"care" + 0.014*"company" + 0.013*"people" + 0.013*"cost" + 0.011*"plan" + 0.011*"bill" + 0.008*"never" + 0.008*"month" + 0.007*"cover" + 0.007*"call"'),
 (1,
  '0.021*"care" + 0.015*"name" + 0.015*"focus" + 0.013*"diagnosis" + 0.012*"country" + 0.012*"pocket" + 0.011*"receive" + 0.011*"payment" + 0.009*"private" + 0.009*"bill"'),
 (2,
  '0.014*"cost" + 0.014*"care" + 0.011*"year" + 0.010*"canada" + 0.009*"european" + 0.009*"people" + 0.007*"government" + 0.007*"treatment" + 0.006*"month" + 0.006*"country"'),
 (3,
  '0.013*"nurse" + 0.011*"work" + 0.006*"grant" + 0.005*"take" + 0.005*"university" + 0.005*"year" + 0.005*"surgery" + 0.005*"pain" + 0.005*"specialist" + 0.005*"like"'),
 (4,
  '0.001*"challenge" + 0.001*"care" + 0.001*"canada" + 0.001*"year" + 0.001*"people" + 0.001*"nurse" + 0.001*"bill" + 0.001*"month" + 0.001*"cost" + 0.001*"country"')]

In [0]:
lda_model_quora = gensim.models.LdaMulticore(bow_corpus_before, num_topics=5, id2word=dictionary_quora, chunksize = 64, eval_every = 5, per_word_topics = 3, minimum_probability = 0.1, passes=25, random_state=23)

In [24]:
lda_model_quora.print_topics()

[(0,
  '0.016*"year" + 0.014*"canada" + 0.012*"cost" + 0.010*"month" + 0.008*"bill" + 0.008*"country" + 0.008*"little" + 0.008*"life" + 0.008*"taxis" + 0.007*"make"'),
 (1,
  '0.017*"nurse" + 0.009*"work" + 0.008*"grant" + 0.008*"university" + 0.007*"year" + 0.006*"take" + 0.006*"pain" + 0.006*"heart" + 0.005*"course" + 0.005*"back"'),
 (2,
  '0.017*"care" + 0.012*"world" + 0.011*"treatment" + 0.010*"canada" + 0.010*"surgeon" + 0.010*"come" + 0.009*"name" + 0.009*"focus" + 0.008*"diagnosis" + 0.008*"renowned"'),
 (3,
  '0.026*"care" + 0.016*"people" + 0.014*"cost" + 0.011*"government" + 0.009*"european" + 0.009*"work" + 0.009*"patient" + 0.007*"want" + 0.007*"company" + 0.007*"year"'),
 (4,
  '0.012*"company" + 0.009*"experience" + 0.009*"month" + 0.009*"visit" + 0.009*"bill" + 0.009*"call" + 0.007*"surgery" + 0.007*"various" + 0.007*"work" + 0.007*"specialist"')]

In [0]:
# df_redUS.to_csv('RedditUSA.csv')

## Preprocessing to remove the most common trivial words

In [0]:
from collections import Counter
s = df_quora['filter V1']
text = s.apply(pd.Series).stack().reset_index(drop=True)
word_counts = Counter(text)
common_words = word_counts.most_common()

In [0]:
df1 = pd.DataFrame(common_words)
df1.to_csv('/content/drive/My Drive/Text Project/df1.csv')

In [0]:
common_stops = ['people','much','well','time', 'would', 'like', 'make', 'work','long','still','even','give', 'good', 'think', 'want', 'right', 'thing', 'year', 'take', 'also', 'know', 'come', 'every', 'life', 'look', 'have', 'will', 'tell', 'believe', 'talk', 'seem', 'since', 'show', 'else','like'] 

In [0]:
new_stopword_list_v1 = ['//www.guardian.com', 'people','much','well','time', 'would', 'like', 'make', 'work','long','still','even','give', 'good', 'think', 'want', 'right', 'thing', 'year', 'take', 'also', 'know', 'come', 'every', 'life', 'look', 'have', 'will', 'tell', 'believe', 'talk', 'seem', 'since', 'show', 'else','like'] 

def additionalStop_V1(comment):
    return [w for w in comment if len(w)>3 and w not in new_stopword_list_v1]

df_quora['filter V2'] = df_quora['filter V1'].map(additionalStop_V1)

In [30]:
df_quora.head()

,Comments,Filter Comments NO punct,filter V1,filter V2
0,People who have experienced both US healthcare...,"[people, experience, us, healthcare, non, -, u...","[people, experience, healthcare, healthcare, w...","[experience, healthcare, healthcare, compare, ..."
1,"As a European, would you prefer the US healthc...","[european, would, prefer, us, healthcare, syst...","[european, would, prefer, healthcare, system, ...","[european, prefer, healthcare, system, brockma..."
2,If you've lived under both the US healthcare s...,"[', have, live, us, healthcare, system, health...","[have, live, healthcare, system, healthcare, s...","[live, healthcare, system, healthcare, system,..."
3,If universal healthcare systems of Europe are ...,"[universal, healthcare, system, europe, great,...","[universal, healthcare, system, europe, great,...","[universal, healthcare, system, europe, great,..."
4,How much does healthcare cost the individual i...,"[much, healthcare, cost, individual, country, ...","[much, healthcare, cost, individual, country, ...","[healthcare, cost, individual, country, univer..."


In [0]:
dictionary_quora = gensim.corpora.Dictionary(df_quora['filter V2'])
dictionary_quora.filter_extremes(no_below=1, no_above=0.5) #, keep_n=100000)
bow_corpus_before = [dictionary_quora.doc2bow(words) for words in df_quora['filter V2']]

In [32]:
lda_model_quora = gensim.models.LdaMulticore(bow_corpus_before, num_topics=5, id2word=dictionary_quora, passes=50, workers=2, chunksize=100, random_state=1000)
lda_model_quora.print_topics()

[(0,
  '0.017*"call" + 0.017*"various" + 0.014*"send" + 0.011*"deal" + 0.011*"network" + 0.011*"sample" + 0.008*"whole" + 0.008*"germany" + 0.008*"suck" + 0.008*"agency"'),
 (1,
  '0.020*"canada" + 0.009*"universal" + 0.006*"coverage" + 0.006*"taxis" + 0.006*"drug" + 0.006*"surgeon" + 0.006*"surgery" + 0.005*"pocket" + 0.005*"name" + 0.005*"focus"'),
 (2,
  '0.001*"challenge" + 0.001*"canada" + 0.001*"european" + 0.001*"government" + 0.001*"surgery" + 0.001*"universal" + 0.001*"nurse" + 0.001*"drug" + 0.001*"different" + 0.001*"taxis"'),
 (3,
  '0.012*"nurse" + 0.008*"pain" + 0.005*"grant" + 0.005*"university" + 0.005*"paperwork" + 0.005*"week" + 0.005*"fill" + 0.005*"office" + 0.005*"specialist" + 0.005*"surgery"'),
 (4,
  '0.019*"european" + 0.019*"government" + 0.013*"patient" + 0.013*"americans" + 0.011*"room" + 0.011*"europe" + 0.010*"heath" + 0.009*"standard" + 0.008*"quality" + 0.007*"however"')]

## Summarization

In [0]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in df_quora['Comments']:
  sentences.append(sent_tokenize(s))
sentences = [y for x in sentences for y in x] # flatten list

In [34]:
sentences[:5]

['People who have experienced both US healthcare and non-US healthcare: how would you compare the care you’ve received (or given) in each healthcare system?Allan Clark, lives in The United States of America (1998-present)Updated Jul 19, 2019 · Author has 145 answers and 50.2k answer viewsOriginally Answered: As someone who has received medical care in both America and another country, how would you compare the American healthcare experience to that of other countries?I’ve been a guest of US, Canada, China, and UK healthcare.China: (focus on cash, then quick care)pay us in cashSee a doctorHe gives you a diagnosis, medication, and your changeCanada: (focus on care)whats your first name?',
 'What are you allergic to?TriageDoctor, diagnosis, prescriptionGive us your last name and social on the way out for our recordsUK: (compassion, care)you’re hurting?',
 'Sorry, dear.',
 'What’s your name?',
 'Where do you live?Let’s sit you down in triageDoctor, empathy, weak drugs for pain, diagnosis, 

In [35]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2019-12-01 03:46:06--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-12-01 03:46:06--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-12-01 03:46:07--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [0]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [0]:
# function to remove stopwords
def remove_stopwords_sentence(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [0]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords_sentence(r.split()) for r in clean_sentences]

In [0]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [0]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [0]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [0]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [45]:
# Extract top 10 sentences as the summary
for i in range(10):
  print(ranked_sentences[i][1])

That visit was less than 5 minutes and it cost me over $500.00 just to squirt a little water in my ear.So no, not only would I not change our healthcare system for the highly overpriced American system, but twice I’ve refused employment transfers to the US specifically because I don’t want to ever have to rely on a US type system.If I lose my job in Canada, I still have the same health coverage.If I develop a ‘pre-existing condition in Canada, I still have the same health coverage.If I have a catastrophic illness or injury in Canada that costs millions to treat, my coverage is not capped.If I show up at the ER with something life-threatening in Canada, I don’t have to wait for treatment while they check what coverage I’ve got and how they’re going to get paid.If I’m in the hospital in Canada at the same time that I’m having a bad month financially, there’s no co-pay or deductible to have to worry about.If there’s some problem with the bill getting paid or questions about what medical c